In [1]:
if True:
    %matplotlib notebook
    figsize = (10,6)
else:
    figsize = (12,8)

from sps_engineering_Lib_dataQuery.databasemanager import DatabaseManager
from matplotlib.dates import num2date, date2num
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

In [2]:
def interpdate(datas):
    
    mintai = np.max([data.tai.min() for data in datas])
    maxtai = np.min([data.tai.max() for data in datas])
    tai = np.arange(mintai, maxtai, 0.0002)
    res = pd.DataFrame({'tai':tai})
    for data in datas:
        for col in data.columns[2:]:
             res[col] = np.interp(res['tai'], data['tai'], data[col])
            
    return res

In [3]:
db = DatabaseManager('tron', 5432, '')

In [4]:
db.init()

<connection object at 0x7f191cae68c8; dsn: 'dbname='archiver' user='pfs' password=xx host='tron' port='5432'', closed: 0>

In [5]:
maxPower = 200

In [6]:
cols= ['val1_%d'%i for i in range(5)] + ['val1_10','val1_11']
colnames = ['DetectorBox', 'Mangin', 'SpiderRodC','ThermalSpreader','FrontRing', 'DetectorStrap1','DetectorStrap2']

temps = []
datasets = []
popts = []
steps = {20: ['2019-01-17 19:00', '2019-01-18 10:00'],
         15: ['2019-01-24 22:00', '2019-01-25 09:00'],
         10: ['2019-01-19 11:00', '2019-01-21 02:00'],
         5: ['2019-01-22 10:50', '2019-01-22 15:00'],
         0 :['2019-01-21 12:00', '2019-01-21 14:50'],
        }
         

In [7]:
specId, datestart, dateend = [1, '2019-02-11 14:30', '2019-02-25 08:00']

steps = {5: ['2019-01-21 22:00', '2019-01-22 10:30']}  #Calib_in

In [8]:
temps = db.dataBetween('xcu_r1__temps', cols=','.join(cols), start=datestart, end=dateend)
cooler = db.dataBetween('xcu_r1__coolertemps', cols='setpoint,tip,power', start=datestart, end=dateend)
temps.columns = ['id','tai'] + colnames
temps['setpoint'] = np.interp(temps['tai'], cooler['tai'], cooler['setpoint'])
temps['tip'] = np.interp(temps['tai'], cooler['tai'], cooler['tip'])
temps['power'] = np.interp(temps['tai'], cooler['tai'], cooler['power'])
#dataset = interpdate([temps, cooler])

In [9]:
temps.drop(columns=['id'], inplace=True)

In [10]:
temps['tai'] = [date.isoformat()[:19] for date in num2date(temps.tai)]
temps.set_index('tai').to_csv('/home/pfs/PFS-R1-Dataset-2019-02-11.csv')

In [ ]:
popt = np.polyfit(dataset.tai, dataset.power, 1)

newtai = np.arange(dataset.tai.min(), dataset.tai.max()+280, 0.0002)
fit = np.polyval(popt, newtai)
[tlim] = np.roots([popt[0], popt[1]-maxPower])

print('reaching maxPower : %s'%(num2date(tlim).isoformat()[:19]))

In [ ]:
popt

In [ ]:
plt.figure(figsize=figsize)
plt.plot_date(dataset.tai, dataset.power, 'bo')
plt.plot_date(dataset.tai, savgol_filter(dataset.power, 7777, 2), '-')
plt.plot_date(newtai, fit, '-')
plt.vlines(tlim, ymin=dataset.power.min(), ymax=200)

plt.grid()

In [ ]:
plt.figure(figsize=figsize)
plt.plot_date(dataset.tai, dataset.power- np.polyval(popt, dataset.tai), '-')
plt.plot_date(dataset.tai, savgol_filter(dataset.power- np.polyval(popt, dataset.tai), 9999, 2), '-')

plt.grid()